# This notebook generates the data defining the case study

A .csv file is created containing:
 - Origin (latitude; longitude; altitude)
 - Destination (latitude; longitude; altitude)
 - Time of departure (formatted; only in seconds)
 - Drone model
 - Purpose
 - Time of planning

For each drone forming the background traffic and surveillance drones traffic.

In [ ]:
from pathlib import Path

"""
We define a default path where the modules developed in USEPE are stored
"""

bluesky_path = Path(Path.home(), 'ws', 'bluesky')

"""
We define a second default path where the scenarios are stored
"""

scenarios_path = Path(Path.home(), 'ws', 'scenarios', 'exercise_3')

In [ ]:
import os
import sys
import configparser
import pickle
import osmnx as ox
import random
import pandas as pd
import datetime

os.chdir(bluesky_path)
from usepe.city_model.utils import read_my_graphml, layersDict
from usepe.city_model.multi_di_graph_3D import MultiDiGrpah3D
from usepe.city_model.city_graph import cityGraph
from usepe.city_model.scenario_definition import createSurveillanceCSV, createATMcsv, createBackgroundTrafficCSV, createScenarioCSV


# Configuration file

We create a config file with the following fields:

[City]

mode = rectangle

hannover_lat = 52.376

hannover_lon = 9.76

zone_size = 1000

hannover_lat_min = 52.412274

hannover_lat_max = 52.466148

hannover_lon_min = 9.638119

hannover_lon_max = 9.745655

import = True

imported_graph_path = data/exe_3_ref.graphml

[Layers]

number_of_layers = 9

layer_width = 25

[Corridors]

corridors = 

altitude = 250

delta_z = 25

speed = 50

acceleration_length = 50

file_path_corridors = usepe/city_model/data/usepe-hannover-corridors.geojson

[Segments]

import = True

path = data/reference_segments.pkl

[BuildingData]

lat_min = 52.412274

lat_max = 52.466148

lon_min = 9.638119

lon_max = 9.745655

divisions = 20

directory_hannover = /home/oda/ws/bluesky/usepe/city_model/data/hannover_building_data

[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2

[Outputs]

graph_path = data/exe_3_ref.graphml

[Strategic_Deconfliction]

ratio = 3

delay = 60


In [ ]:
 # -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = Path(scenarios_path, 'settings_exe_3_ref.cfg')
config = configparser.ConfigParser()
config.read( config_path )

In [ ]:
# -------------- 2. CITY GRAPH -------------------------
"""
This section creates a city graph or loads the graph defined in the 'City' section of the
configuration file.
"""
# City
os.chdir(scenarios_path)
if config['City'].getboolean( 'import' ):
    filepath = config['City']['imported_graph_path']
    G = read_my_graphml( filepath )
    G = MultiDiGrpah3D( G )
    # fig, ax = ox.plot_graph( G )
else:
    print('Check the path to the graph file')
    G = cityGraph( config )

In [ ]:
# -------------- 3. SEGMENTS ----------------------------
"""
This section creates an airspace segmentation or loads the segmentation defined in the 'Segments'
section of the configuration file.
Comment it to neglect the segmentation
"""
os.chdir(scenarios_path)
if config['Segments'].getboolean( 'import' ):
    segments = pd.read_json(config['Segments']['imported_segments_file_path'], orient="records", lines=True )
else:
    segments = None

# Data for the experiments

The densities to be tested in the experiments are
 - Low: 0.0964 simultaneous flights / km^2
 - Medium: 0.964 simultaneous flights / km^2
 - High: 9.64 simultaneous flights / km^2
 
The average flight duration calculated is: 1000 s

The simulation time considered is: 1h (3600 s)


In [ ]:
# General
simulation_time = 3600

# Background Traffic

In [ ]:
# Background traffic
density = {'Low': 0.0964, 'Medium': 0.964, 'High': 9.64}

background_traffic = {'density': density['High'],
                      'avg_flight_duration': 1000,
                      'simulation_time': simulation_time,
                      'G': G,
                      'segments': segments,
                      'config': config,
                      'default_path': bluesky_path}

# os.chdir(bluesky_path)
# Add background traffic
# print('Creating background traffic...')
# createBackgroundTrafficCSV(density['Medium'], avg_flight_duration, simulation_time, G, segments, config, bluesky_path)

# Surveillance operation

In [ ]:
os.chdir(scenarios_path)
origins = [(52.44893, 9.72996, 5),
           (52.43906, 9.69446, 5),
           (52.42035, 9.65075, 51),
           (52.42035, 9.65075, 51),
           (52.43218, 9.68395, 5)]

destinations = [(52.45044, 9.71846, 50),
                (52.44066, 9.68234, 50),
                (52.44252, 9.67161, 50),
                (52.43126, 9.67879, 50),
                (52.43572, 9.68334, 50)]

departure_times = [60, 60, 60, 120, 315]

drone_models = ['M600', 'M600', 'M600', 'M600', 'W178']

operation_ids = ['op1', 'op2', 'op3', 'op4', 'op5']

operation_durations = [600, 450, 330, 240, 660]

surveillance = {'origins': origins,
                'destinations': destinations,
                'departure_times': departure_times,
                'drone_models': drone_models,
                'operation_ids': operation_ids,
                'operation_durations': operation_durations,
                'simulation_time': simulation_time}

# Add operation scenario
# print('Creating surveillance drones...')
# createSurveillanceCSV(origins, destinations, departure_times, drone_models, operation_ids, operation_durations, simulation_time)

# ATM Operations

In [ ]:
os.chdir(scenarios_path)
origins = [(52.46090, 9.68511, 0),
           (52.458984, 9.564432, 0),
           (52.454977, 9.677607, 0),
           (52.467712, 9.664708, 0)]

departure_times = [180, 60, 60, 60]

aircraft_models = ['EC35', 'A320', 'A320', 'B738']

operation_ids = ['helicopter', 'eurowings', 'lufthansa', 'turkish']

atm = {'origins': origins,
       'departure_times': departure_times,
       'aircraft_models': aircraft_models,
       'operation_ids': operation_ids}

# createATMcsv(origins, departure_times, aircraft_models)

# Combined flight plans

In [ ]:
file_name = 'EXE-3-High'

nPlans = 10

createScenarioCSV(file_name, nPlans, background_traffic=background_traffic, surveillance=surveillance, atm=atm)